In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')


In [2]:
# import os
# os.chdir("/content/gdrive/MyDrive/Gaussian2D")


In [ ]:
# pwd


In [ ]:
# !nvidia-smi


In [5]:
import os
import cv2
import imageio
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from random import randint
# from utils.loss_utils import l1_loss, ssim
# from gaussian_renderer import render, network_gui
import sys
# from scene import Scene, GaussianModel
# from utils.general_utils import safe_state
import uuid
from tqdm import tqdm
# from utils.image_utils import psnr
from argparse import ArgumentParser, Namespace
# from arguments import ModelParams, PipelineParams, OptimizationParams

from Gaussian2D import Gaussian2DModel

try:
    from torch.utils.tensorboard import SummaryWriter
    TENSORBOARD_FOUND = True
except ImportError:
    TENSORBOARD_FOUND = False

import time



In [6]:
imgPathList = [
    "./Pics/Pikachu.png",      # 皮卡丘 600*600
    "./Pics/Charmander.png",   # 小火龙 600*600
    "./Pics/Squirtle.png",     # 杰尼龟 600*600
    "./Pics/Bulbasaur.png",    # 妙蛙种子 600*600
    "./Pics/Lapras.png",       # 乘龙 600*600
    "./Pics/Meowth.png",       # 喵喵 600*600
    "./Pics/Mewtwo.png",       # 梦幻  600*600
    "./Pics/Rayquaza.png",     # 裂空座 1000*1000
    "./Pics/Groudon.png"      # 固拉多  1000*1000
]


In [7]:
def createGIF(dirPath):
    imageNameList = [f for f in os.listdir(dirPath) if f.endswith('.png')]
    frames = []
    for imageName in imageNameList:
        imgPath = os.path.join(dirPath, imageName)
        imgBGR = cv2.imread(imgPath, cv2.IMREAD_COLOR)
        imgRGB = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB)
        frames.append(imgRGB)

    gifName = f'output_{len(frames)}_frames.gif'
    gifPath = os.path.join(dirPath, gifName)
    imageio.mimsave(gifPath, frames, duration=0.25)  # duration 是每帧的显示时间 (s)
    print(f'GIF saved at: {gifPath}')


In [8]:
def train(imgPath, Epoch, numGaussian=800, saveRenderInterval=20, densifyInterval=None, densifyAdd=None):

    startTime = time.time()
    startTimeInt = int(startTime)

    imgBGR = cv2.imread(imgPath, flags=cv2.IMREAD_COLOR)
    imgRGB = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB)
    imgNormalizedRGB = imgRGB/255.0
    imgNormalizedBGR = imgBGR/255.0


    # cv2.imshow('imgNormalized', imgNormalized)
    # cv2.waitKey(100)

    # fig =plt.figure(figsize=(10, 5))  # 设置图像大小
    # plt.suptitle('Gaussian 2D Model Visualize')  # 设置窗口标题
    # fig.canvas.manager.set_window_title('Gaussian 2D Model Visualize')  # 设置窗口标题

    imgShape = imgNormalizedRGB.shape
    print(imgShape)

    gaussian = Gaussian2DModel(numGaussian, imgNormalizedBGR)

    avatarName = os.path.splitext(os.path.basename(imgPath))[0]
    fileDirPath = rf"./output/{startTimeInt}_{Epoch}_Epoch_{numGaussian}_Gaussians_{avatarName}"
    if densifyInterval is not None:
      fileDirPath = fileDirPath+"_Densify"

    if not os.path.exists(fileDirPath):
        os.mkdir(fileDirPath)
    logDirPath = f'./log/{startTimeInt}_{Epoch}_Epoch_{numGaussian}_Gaussians_{avatarName}'
    if not os.path.exists(logDirPath):
        os.mkdir(logDirPath)
    tensorboardWriter = SummaryWriter(logDirPath)

    numEpochs = Epoch
    for epoch in range(numEpochs):
        loss = gaussian.lossAndBackwardAndStep()
        tensorboardWriter.add_scalar('Loss', loss, epoch)

        if epoch % saveRenderInterval == 0:
            print(f'Epoch {epoch}, Loss: {loss}')
            # imgTorch = gaussian.render()
            imgRenderBGR = (gaussian.renderImgTorch).detach().cpu().numpy().copy()  # detach() 方法会创建一个新的张量，断开与计算图的连接
            imgRenderRGB = cv2.cvtColor(imgRenderBGR, cv2.COLOR_BGR2RGB)
            imgRenderBGR = (imgRenderBGR*255).astype(np.int32)


            fileName = rf"Epoch_{epoch:04}_Loss_{loss:08.5f}_Gaussians_{gaussian.numGaussians}.png"

            filePath = os.path.join(fileDirPath, fileName)
            absolutePath = os.path.abspath(filePath)
            print("absolutePath : ", absolutePath)

            cv2.imwrite(filePath, imgRenderBGR)
            tensorboardWriter.add_image('Rendered Image', np.transpose(imgRenderRGB, (2, 0, 1)), epoch)

        if epoch % densifyInterval == 0:
            gaussian.densifyGaussianAtBigDiffList(pickTopDiffNums = densifyAdd)


            # imgRenderRGB = cv2.cvtColor(imgRenderBGR, cv2.COLOR_BGR2RGB)

            # plt.clf()

            # plt.subplot(1, 2, 1)  # 1行2列，当前为第1个
            # plt.imshow(imgNormalizedRGB)
            # plt.axis('off')  # 关闭坐标轴
            # plt.title('Original Image')

            # plt.subplot(1, 2, 2)  # 1行2列，当前为第1个
            # # plt.gcf().canvas.setWindowTitle(f'Epoch: {epoch}, Loss: {loss:.4f}')  # 设置窗口标题
            # plt.imshow(imgRenderRGB)
            # plt.axis('off')  # 关闭坐标轴
            # plt.title(f'Epoch: {epoch}, Loss: {loss:.4f}')

            # plt.show()
            # plt.pause(0.1)  # 暂停以更新图像

    tensorboardWriter.close()
    return fileDirPath

    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


In [ ]:
# train(imgPath=imgPathList[0], Epoch=1000, numGaussian=250)
# train(imgPath=imgPathList[0], Epoch=1000, numGaussian=500)
# train(imgPath=imgPathList[0], Epoch=1000, numGaussian=750)
# train(imgPath=imgPathList[0], Epoch=1000, numGaussian=1000)

# fileDirPath = train(imgPath=imgPathList[0], Epoch=1000, numGaussian=250, saveRenderInterval=20)
# createGIF(fileDirPath)
# fileDirPath = train(imgPath=imgPathList[5], Epoch=1000, numGaussian=500, saveRenderInterval=20)
# createGIF(fileDirPath)
# fileDirPath = train(imgPath=imgPathList[0], Epoch=1000, numGaussian=750, saveRenderInterval=20)
# createGIF(fileDirPath)
fileDirPath = train(imgPath=imgPathList[5], Epoch=800, numGaussian=500, saveRenderInterval=20, densifyInterval=100, densifyAdd=50)
createGIF(fileDirPath)


In [10]:
# plt.figure(figsize=(10, 5))  # 设置图像大小
# plt.suptitle('Gaussian 2D Model Visualize')  # 设置窗口标题
# plt.subplot(1, 2, 1)  # 1行2列，当前为第1个
# plt.imshow(imgNormalizedRGB)
# plt.axis('off')  # 关闭坐标轴
# plt.title('Original Image')

# plt.subplot(1, 2, 2)  # 1行2列，当前为第1个
# # plt.gcf().canvas.setWindowTitle(f'Epoch: {epoch}, Loss: {loss:.4f}')  # 设置窗口标题
# plt.imshow((gaussian.renderImgTorch.detach().cpu().numpy()*255).astype(np.int32))
# plt.axis('off')  # 关闭坐标轴
# plt.title(f'Epoch: {epoch}, Loss: {loss:.4f}')

# plt.show()
# plt.pause(0.1)  # 暂停以更新图像


In [ ]:
fileDirPath = train(imgPath=imgPathList[0], Epoch=800, numGaussian=500, saveRenderInterval=20, densifyInterval=100, densifyAdd=50)
createGIF(fileDirPath)
